In [6]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler, RobustScaler
import matplotlib.pyplot as plt
import json
from config import LOOKBACK_L, HORIZON_H, TEST_SAMPLES, FEATURES

In [7]:
print(f"""
📊 Model Configuration
────────────────────────────
LOOKBACK_L : {LOOKBACK_L}
HORIZON_H  : {HORIZON_H}
TEST_SAMPLES: {TEST_SAMPLES}
FEATURES    : {FEATURES}
────────────────────────────
""")


📊 Model Configuration
────────────────────────────
LOOKBACK_L : 48
HORIZON_H  : 3
TEST_SAMPLES: 2000
FEATURES    : ['price_pct_change', 'volume_pct_change']
────────────────────────────



In [8]:
model = tf.keras.models.load_model("./results/models/lstm.regression.keras")

In [9]:
eth_data = pd.read_csv('./paper_wallet_data/new_eth_hourly_data.csv')
eth_data = eth_data.dropna().reset_index(drop=True)
eth_data.head()

,time,low,high,open,close,volume
0,2024-08-15 07:00:00+00:00,2589.39,2625.01,2623.84,2614.76,3283.535428
1,2024-08-15 08:00:00+00:00,2606.05,2622.11,2615.07,2621.35,1855.089703
2,2024-08-15 09:00:00+00:00,2617.40,2631.12,2621.54,2623.63,1193.382931
3,2024-08-15 10:00:00+00:00,2614.40,2627.65,2623.74,2618.11,760.890678
4,2024-08-15 11:00:00+00:00,2617.64,2643.82,2618.31,2637.94,1485.566206


In [10]:
eth_data['time'] = pd.to_datetime(eth_data['time'], utc=True)

eth_data["hour"] = eth_data["time"].dt.hour

In [11]:
eth_data['price_pct_change'] = eth_data['close'].pct_change(1)   
eth_data['volume_pct_change'] = eth_data['volume'].pct_change(1)
eth_data = eth_data.dropna().reset_index(drop=True)
hours = eth_data['hour']
hours = np.array(hours, dtype=np.int32)   # force integers 0–23

y = eth_data['close'].pct_change(HORIZON_H)

In [12]:
#get ETH id
with open("paper_wallet_data/asset_to_id.json", "r") as f:
    asset_to_id = json.load(f)

# Example usage
eth_id = asset_to_id["ETH"]
print(f"ETH Id: {eth_id}")

ETH Id: 8


In [13]:
def make_sequences(df, L):
    X_list = []
    X = df[FEATURES].values
    X = X
    for t in range(L, len(df)):
            X_list.append(X[t-L:t,:])
    return np.stack(X_list[-TEST_SAMPLES:], dtype=np.float64)

X = make_sequences(eth_data, LOOKBACK_L)

In [14]:
A = np.full((TEST_SAMPLES,), eth_id, dtype=np.int32)

In [15]:
yhat = model.predict({"Returns": X, "Tickers": A})
print(yhat.shape)  # (1000, 1)

63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
(2000, 1)


In [16]:
def compute_true_targets(prices, horizon=3, n_samples=TEST_SAMPLES):
    """
    Compute true forward % change targets for the last n_samples windows.
    
    prices: array-like of raw prices
    horizon: prediction horizon (steps ahead)
    n_samples: how many sequences we predicted
    seq_len: length of each input window (48 in your case)
    """
    prices = np.asarray(prices, dtype=np.float32)
    
    # forward return for every valid index
    forward = (prices[horizon:] - prices[:-horizon]) / prices[:-horizon]
    
    # slice the last n_samples, aligned with your X windows
    true_vals = forward[-n_samples:]
    
    return true_vals

prices = eth_data['close']
true_changes = compute_true_targets(prices, horizon=3, n_samples=TEST_SAMPLES)

In [17]:
# Metrics
mae = np.mean(np.abs(yhat - true_changes))
#corr = np.corrcoef(yhat, true_changes)[0,1]
directional_acc = np.mean(np.sign(yhat) == np.sign(true_changes))

print(f"MAE: {mae:.4f},")

MAE: 0.0097,


In [18]:
y_count = np.sum(yhat < 0)
print(f"Predicted Up predictions: {len(yhat) - y_count} Down predictions: {y_count}")
true_count = np.sum(true_changes < 0)
print(f"Real Up predictions: {len(true_changes) - true_count} Down predictions: {true_count}")

Predicted Up predictions: 1441 Down predictions: 559
Real Up predictions: 1029 Down predictions: 971


In [19]:
correct = 0
for i in range(len(yhat)):
    if (yhat[i] < 0 and true_changes[i] < 0) or (yhat[i] > 0 and true_changes[i] > 0):
        correct += 1

print(f"Correct direction: {correct} out of {len(yhat)}")

Correct direction: 1456 out of 2000


In [20]:
yhat[:10]

array([[ 0.00341166],
       [-0.00295093],
       [-0.0084643 ],
       [ 0.00151799],
       [ 0.00174111],
       [-0.00616351],
       [ 0.00102671],
       [ 0.00651019],
       [ 0.00486992],
       [ 0.00486412]], dtype=float32)

In [21]:
true_changes[:10]

array([-0.00721127, -0.01270129, -0.01021208, -0.00702927, -0.00752429,
       -0.00810989, -0.00129622,  0.00717669,  0.00445129,  0.00318476],
      dtype=float32)